# Calculate sidewalk width
Using centerline extraction (also called skeleton line, axis line, or medial line extraction)

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pathlib
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import shapely.ops as so

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils

import upc_sw.poly_utils as poly_utils

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
# Optional, for parallel processing of geodataframes.
import dask_geopandas as dgpd
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [ ]:
# Paths
pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/output/'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992' #local crs

# Whether to merge sidewalks before segmentation and width computation
merge_sidewalks = True

# Tolerance for centerline simplification
simplify_tolerance = 0.2

# Min area size of a sidewalk polygon in sqm for which width will be computed
min_area_size = 1

# Max linestring length in meters
max_line_length = 5

# Resolution (in m) for min and avg width computation
width_resolution = 1

# Precision (in decimals) for min and avg width computation
width_precision = 1

## Read the sidewalk and obstacle data

In [ ]:
# Read sidewalk with obstacle data
obstacle_file = f'{out_folder}/sidewalks_with_obstacles.gpkg'
df = gpd.read_file(obstacle_file, geometry='geometry', crs=CRS)

if merge_sidewalks:
    # Merge sidewalk polygons
    df = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]), crs=CRS)
else:
    # Explode MultiPolygons into their parts
    df = df.explode(index_parts=False)

# Ignore sidewalk polygons that are too small
df = df[df.area > min_area_size]

## Calculate width along centerline segments

In [ ]:
# TODO: keep sidewalk ID along with segments for back-reference?

def get_segments_width(row):
    # Get centerlines.
    cl = poly_utils.get_centerlines(row.geometry)
    # Merge linestrings.
    cl = so.linemerge(cl)
    # Remove short line ends and dead-ends.
    cl = poly_utils.remove_short_lines(cl, max_line_length)
    # Simplify lines.
    cl = cl.simplify(simplify_tolerance, preserve_topology=True)
    # Segment lines and compute avg and min width per segment.
    segments = poly_utils.get_segments(cl)
    avg_width, min_width = poly_utils.get_avg_width(
                    row.geometry, segments, width_resolution, width_precision)
    return {'segments': segments, 'avg_width': avg_width, 'min_width': min_width}

### Single thread

In [ ]:
# Single thread, can be slow for big dataframes.

# if you get an error here, make sure you use tqdm>=4.61.2
segment_df = pd.DataFrame(df.progress_apply(get_segments_width, axis=1).values.tolist())

### Parallel

In [ ]:
# Alternative: parallel processing for big dataframes.

# TODO: sensible (or variable) setting for npartitions
ddf = dgpd.from_geopandas(df, npartitions=24)

In [ ]:
segment_df = pd.DataFrame(
                    (ddf.apply(get_segments_width, axis=1, meta=(object))
                     .compute().values.tolist()))

### Explode into individual segments

In [ ]:
segment_df = pd.concat([gpd.GeoDataFrame({'geometry': row.segments,
                                           'avg_width': row.avg_width,
                                           'min_width': row.min_width})
                         for _, row in segment_df.iterrows()])
segment_df.set_crs(crs=CRS, inplace=True);

## Check coverage of point cloud data on sidewalks

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}run1/')
all_tiles_poly = so.unary_union([poly_utils.tilecode_to_poly(tile) for tile in all_tiles])

segment_df['pc_coverage'] = segment_df.intersects(all_tiles_poly)

## Store output

In [ ]:
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

segments_file = f'{out_folder}sidewalk_segments.gpkg'
segment_df.to_file(segments_file, driver='GPKG')

## Plot results

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt


tilecodes = all_tiles

plot_shape = so.unary_union([poly_utils.tilecode_to_poly(tilecode) for tilecode in tilecodes])
(x_min, y_min, x_max, y_max) = plot_shape.bounds
df_plot = df[df.intersects(plot_shape)]

fig, ax = plt.subplots(1, figsize=(6,6))

df_plot.set_geometry('geometry').plot(ax=ax, color='grey', alpha=0.5)
segment_df.plot(ax=ax, column='min_width', cmap='Spectral', vmin=0, vmax=10, legend=True)

ax.set_xlim([x_min, x_max])
ax.set_ylim([y_min, y_max])

plt.show()